# Customize output format in prompt

## User prompt configuration

### [X] Create prompt template

The prompt template is defined in the file:

[/workspace/data/prompts/weather/template.md](/workspace/data/prompts/weather/template.md)

And it will be loaded as follows:

In [2]:
folder_prompt_name = 'weather' #TODO: define your folder

In [3]:
from pathlib import Path
folder = Path(f'/workspace/data/prompts/{folder_prompt_name}')

path = folder / 'template.md'
with open(path, 'r') as file:
    template = file.read()

template

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/data/prompts/weather/template.md'

### [X] Define custom output format and import it

[/workspace/data/prompts/weather/output_format.py](/workspace/data/prompts/weather/output_format.py)

In [3]:
output_class_name = 'WeatherList' #TODO: define your class

In [4]:
from importlib import import_module
OutputParser = getattr(import_module(f'data.prompts.{folder_prompt_name}.output_parser'), output_class_name)

## Combine template and output format

In [5]:
from modules.prompt import CustomPrompt

custom_prompt = CustomPrompt(template, OutputParser)
prompt = custom_prompt.get_prompt()
prompt

PromptTemplate(input_variables=['DATE_END', 'DATE_START', 'LOCATION'], input_types={}, partial_variables={'FORMAT_INSTRUCTIONS': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Weather": {"properties": {"location": {"description": "location of the weather", "title": "Location", "type": "string"}, "date": {"description": "date of the weather", "format": "date", "title": "Date", "type": "string"}, "description": {"description": "description of the weather", "title": "Description", "type": "string"}, "temperature_min": {"description": "minimum temperature of the weather", "

## Chain

### Define model

In [6]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff41688aa0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff4168b380>, root_client=<openai.OpenAI object at 0xffff41e19bb0>, root_async_client=<openai.AsyncOpenAI object at 0xffff41688b00>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

### Compose chain

In [7]:
chain = prompt | model | custom_prompt.parser
chain

PromptTemplate(input_variables=['DATE_END', 'DATE_START', 'LOCATION'], input_types={}, partial_variables={'FORMAT_INSTRUCTIONS': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Weather": {"properties": {"location": {"description": "location of the weather", "title": "Location", "type": "string"}, "date": {"description": "date of the weather", "format": "date", "title": "Date", "type": "string"}, "description": {"description": "description of the weather", "title": "Description", "type": "string"}, "temperature_min": {"description": "minimum temperature of the weather", "

### [X] Invoke chain

To get the response, we need to invoke the chain with the input data.

In [8]:
output = chain.invoke({
    'LOCATION': 'New York',
    'DATE_START': '2025-04-12',
    'DATE_END': '2025-04-18'
})

## Output

### JSON

In [9]:
data = output.model_dump()
data

{'weathers': [{'location': 'New York, NY, United States',
   'date': datetime.date(2025, 4, 12),
   'description': 'Cold with rain and drizzle, becoming windy this afternoon',
   'temperature_min': 40.0,
   'temperature_max': 42.0,
   'temperature_avg': 41.0,
   'url': 'https://www.weather.com/weather/today/l/USNY0996:1:US'},
  {'location': 'New York, NY, United States',
   'date': datetime.date(2025, 4, 13),
   'description': 'Areas of drizzle in the morning; otherwise, warmer with clouds followed by a brightening sky',
   'temperature_min': 46.0,
   'temperature_max': 57.0,
   'temperature_avg': 51.5,
   'url': 'https://www.weather.com/weather/today/l/USNY0996:1:US'},
  {'location': 'New York, NY, United States',
   'date': datetime.date(2025, 4, 14),
   'description': 'Milder with clouds and breaks of sun',
   'temperature_min': 51.0,
   'temperature_max': 64.0,
   'temperature_avg': 57.5,
   'url': 'https://www.weather.com/weather/today/l/USNY0996:1:US'},
  {'location': 'New York, 

### DataFrame

In [10]:
import pandas as pd

data_values = list(data.values())[0]
df = pd.DataFrame(data_values)
df.style

,location,date,description,temperature_min,temperature_max,temperature_avg,url
0,"New York, NY, United States",2025-04-12,"Cold with rain and drizzle, becoming windy this afternoon",40.000000,42.000000,41.000000,https://www.weather.com/weather/today/l/USNY0996:1:US
1,"New York, NY, United States",2025-04-13,"Areas of drizzle in the morning; otherwise, warmer with clouds followed by a brightening sky",46.000000,57.000000,51.500000,https://www.weather.com/weather/today/l/USNY0996:1:US
2,"New York, NY, United States",2025-04-14,Milder with clouds and breaks of sun,51.000000,64.000000,57.500000,https://www.weather.com/weather/today/l/USNY0996:1:US
3,"New York, NY, United States",2025-04-15,Increasingly windy with decreasing clouds; a shower cannot be ruled out,42.000000,64.000000,53.000000,https://www.weather.com/weather/today/l/USNY0996:1:US
4,"New York, NY, United States",2025-04-16,Cooler with increasing amounts of sunshine,40.000000,53.000000,46.500000,https://www.weather.com/weather/today/l/USNY0996:1:US
5,"New York, NY, United States",2025-04-17,Sunny and breezy,44.000000,58.000000,51.000000,https://www.weather.com/weather/today/l/USNY0996:1:US
6,"New York, NY, United States",2025-04-18,Mostly cloudy with a couple of showers possible,50.000000,60.000000,55.000000,https://www.weather.com/weather/today/l/USNY0996:1:US


### Export to Excel and CSV

In [11]:
from datetime import datetime
import os

# Create a folder with the current datetime
current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
output_folder = folder / f'outputs/{current_datetime}'
output_folder.mkdir(parents=True, exist_ok=True)

# Save the files in the newly created folder
df.to_excel(output_folder / 'output.xlsx', index=False)
df.to_csv(output_folder / 'output.csv', index=False)